# NLP – Text Classification

# 🗣️ NLP – Text Classification (LSTM)

## Dữ liệu: ../data/nlp_texts.csv (hoặc trong Drive)

In [ ]:
!pip install -U tensorflow keras pandas scikit-learn

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

drive_path = '/content/drive/MyDrive/keras_project/data/nlp_texts.csv'
path = drive_path if tf.io.gfile.exists(drive_path) else '../data/nlp_texts.csv'
df = pd.read_csv(path)
texts = df['text'].astype(str).tolist()
labels = df['label'].values.astype('float32')

max_tokens=5000; max_len=50
vec = layers.TextVectorization(max_tokens=max_tokens, output_sequence_length=max_len)
vec.adapt(tf.data.Dataset.from_tensor_slices(texts).batch(32))

X = vec(np.array([[t] for t in texts])); y = labels
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = keras.Sequential([
    layers.Embedding(max_tokens, 64, input_length=max_len),
    layers.LSTM(64),
    layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
hist = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=8, batch_size=32)

plt.figure(); plt.plot(hist.history['accuracy'], label='acc'); plt.plot(hist.history['val_accuracy'], label='val_acc'); plt.legend(); plt.title('Accuracy'); plt.xlabel('epoch'); plt.show()
samples = ['Keras training is great', 'This model is terrible']
Xs = vec(np.array([[t] for t in samples])); probs = model.predict(Xs).squeeze()
for s,p in zip(samples, probs): print(s, '->', float(p))
